# GFortran AArch64 minimal

2025-06-12

For this exercise, we've managed to create an incredibly tiny AArch64 program, just 20 bytes in size, even though we started writing it in Fortran. It's not a standard executable file that your operating system would recognize (like an ELF file); instead, it's a raw, bare-metal binary that needs a special, custom loader to make it run.

In [1]:
! gfortran --version

GNU Fortran (Ubuntu 13.3.0-6ubuntu2~24.04) 13.3.0
Copyright (C) 2023 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [2]:
! as --version

GNU assembler (GNU Binutils for Ubuntu) 2.42
Copyright (C) 2024 Free Software Foundation, Inc.
This program is free software; you may redistribute it under the terms of
the GNU General Public License version 3 or later.
This program has absolutely no warranty.
This assembler was configured for a target of `aarch64-linux-gnu'.


In [77]:
! ld --version

GNU ld (GNU Binutils for Ubuntu) 2.42
Copyright (C) 2024 Free Software Foundation, Inc.
This program is free software; you may redistribute it under the terms of
the GNU General Public License version 3 or (at your option) a later version.
This program has absolutely no warranty.


In [4]:
! gcc --version

gcc (Ubuntu 13.3.0-6ubuntu2~24.04) 13.3.0
Copyright (C) 2023 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [65]:
%%writefile entry.s
/*
 * entry.s
 *
 * This is the actual entry point of our executable for Linux AArch64.
 * It's crucial for setting up the minimal environment before your Fortran code runs.
 *
 */
    .global _start  // Makes the _start label visible to the linker. This is where execution begins.
    .text           // Places the following code in the executable .text section.

_start:
    /* 1. Call our Fortran function.
     * 'bl' (Branch with Link) saves the return address in the Link Register (lr/x30).
     */
    bl        fortran_main            // Call the fortran_main function. Its return value will be placed in x0/w0.

    /* 2. Prepare for the 'exit' system call.
     * The return value from 'fortran_main' is already correctly in x0/w0, which is
     * where the 'exit' syscall expects its status argument.
     */

    /* Put the syscall number for 'exit' (93) into register w8. */
    mov       w8, #93                 // Linux AArch64 syscall convention: syscall number goes into w8/x8.

    /* 4. Invoke the kernel to perform the system call. */
    svc       #0                      // 'Supervisor Call' instruction. This traps into the kernel, which then executes the 'exit' syscall based on w8 and x0.

Overwriting entry.s


In [66]:
! as entry.s -o entry.o

Why a FUNCTION instead of a PROGRAM?
-----------------------------------

1. **External Control**: A standard Fortran PROGRAM block is designed to be
   the primary entry point for a standalone Fortran application. It has
   its own built-in startup and shutdown routines managed by the Fortran
   runtime library.
3. **Minimal Executable Size**: Our goal is to create the smallest possible
   executable by writing our own `_start` entry point in Assembly. This
   Assembly code handles the very basic setup (like stack alignment) and
   then directly calls a specific function.
4. **Avoiding Fortran Runtime Overhead**: If we used a `PROGRAM` block,
  the gfortran compiler would automatically link in much more of the
   Fortran runtime library (libgfortran.a) and its own startup code.
   This would significantly increase the executable size, which goes
   against our goal of creating a "tiny" executable.
5. **C Interoperability (bind(C))**: By defining `fortran_main` as a
   `FUNCTION` and using `bind(C, name='fortran_main')`, we tell gfortran
   to create a function that follows the C calling conventions. This means
   its name in the object file will be exactly `fortran_main` (without
   any Fortran-specific mangling like underscores or case changes), making
  it easy for our Assembly `_start` routine to call it directly.


In [67]:
%%writefile tiny_main.f90
! This is our Fortran "payload".
! It's not a PROGRAM, but a FUNCTION, so it can be called from elsewhere.
! We use bind(C) to ensure the symbol name in the object file
! is exactly "fortran_main", without any suffixes or case changes
! that gfortran might otherwise add.

integer function fortran_main() bind(C, name='fortran_main')
    implicit none  ! Requires all variables to be explicitly declared.
    fortran_main = 42 ! Assigns the value 42 as the return value of the function.
end function fortran_main

Overwriting tiny_main.f90


In [87]:
! gfortran -Os -c tiny_main.f90 -o tiny_main.o

In [88]:
! ld -o a.out entry.o tiny_main.o --nmagic --strip-all

In [89]:
! strip -R .eh_frame -R.comment a.out

In [90]:
! ./a.out ; echo $?

42


In [91]:
! wc -c a.out

408 a.out


In [92]:
! file a.out

a.out: ELF 64-bit LSB executable, ARM aarch64, version 1 (SYSV), statically linked, stripped


In [93]:
! size a.out

   text	   data	    bss	    dec	    hex	filename
     20	      0	      0	     20	     14	a.out


In [94]:
! objdump -d a.out


a.out:     file format elf64-littleaarch64


Disassembly of section .text:

00000000004000b0 <.text>:
  4000b0:	94000003 	bl	0x4000bc
  4000b4:	52800ba8 	mov	w8, #0x5d                  	// #93
  4000b8:	d4000001 	svc	#0x0
  4000bc:	52800540 	mov	w0, #0x2a                  	// #42
  4000c0:	d65f03c0 	ret


In [95]:
! objdump -h a.out


a.out:     file format elf64-littleaarch64

Sections:
Idx Name          Size      VMA               LMA               File off  Algn
  0 .text         00000014  00000000004000b0  00000000004000b0  000000b0  2**2
                  CONTENTS, ALLOC, LOAD, READONLY, CODE


In [96]:
! objdump -s a.out


a.out:     file format elf64-littleaarch64

Contents of section .text:
 4000b0 03000094 a80b8052 010000d4 40058052  .......R....@..R
 4000c0 c0035fd6                             .._.            


In [97]:
! objdump -d a.out


a.out:     file format elf64-littleaarch64


Disassembly of section .text:

00000000004000b0 <.text>:
  4000b0:	94000003 	bl	0x4000bc
  4000b4:	52800ba8 	mov	w8, #0x5d                  	// #93
  4000b8:	d4000001 	svc	#0x0
  4000bc:	52800540 	mov	w0, #0x2a                  	// #42
  4000c0:	d65f03c0 	ret


In [98]:
! hexdump -C a.out

00000000  7f 45 4c 46 02 01 01 00  00 00 00 00 00 00 00 00  |.ELF............|
00000010  02 00 b7 00 01 00 00 00  b0 00 40 00 00 00 00 00  |..........@.....|
00000020  40 00 00 00 00 00 00 00  d8 00 00 00 00 00 00 00  |@...............|
00000030  00 00 00 00 40 00 38 00  02 00 40 00 03 00 02 00  |....@.8...@.....|
00000040  01 00 00 00 05 00 00 00  b0 00 00 00 00 00 00 00  |................|
00000050  b0 00 40 00 00 00 00 00  b0 00 40 00 00 00 00 00  |..@.......@.....|
00000060  14 00 00 00 00 00 00 00  14 00 00 00 00 00 00 00  |................|
00000070  08 00 00 00 00 00 00 00  51 e5 74 64 06 00 00 00  |........Q.td....|
00000080  00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  |................|
*
000000a0  00 00 00 00 00 00 00 00  08 00 00 00 00 00 00 00  |................|
000000b0  03 00 00 94 a8 0b 80 52  01 00 00 d4 40 05 80 52  |.......R....@..R|
000000c0  c0 03 5f d6 00 2e 73 68  73 74 72 74 61 62 00 2e  |.._...shstrtab..|
000000d0  74 65 78 74 00 00 00 00  00 00 00 00 00 

It would be possible to continue and reduce the size of the executable even further, but it starts to affect the binutils tools too much, making it difficult to view and debug, which is not the objective in this case. The size obtained of 408 Bytes for the ELF executable is already an impressive size.

Going further, our ELF executable program in Fortran reaches 196 bytes, but the binutils tools start to generate little or no information:

In [55]:
! strip --strip-section-headers a.out

In [56]:
! ./a.out ; echo $?

42


In [57]:
! wc -c a.out

196 a.out


In [58]:
! size a.out

   text	   data	    bss	    dec	    hex	filename
      0	      0	      0	      0	      0	a.out


In [59]:
! objdump -d a.out


a.out:     file format elf64-littleaarch64



In [60]:
! objdump -h a.out


a.out:     file format elf64-littleaarch64

Sections:
Idx Name          Size      VMA               LMA               File off  Algn


In [61]:
! objdump -s a.out


a.out:     file format elf64-littleaarch64



In [63]:
! objdump -d a.out


a.out:     file format elf64-littleaarch64



In [62]:
! hexdump -C a.out

00000000  7f 45 4c 46 02 01 01 00  00 00 00 00 00 00 00 00  |.ELF............|
00000010  02 00 b7 00 01 00 00 00  b0 00 40 00 00 00 00 00  |..........@.....|
00000020  40 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  |@...............|
00000030  00 00 00 00 40 00 38 00  02 00 00 00 00 00 00 00  |....@.8.........|
00000040  01 00 00 00 05 00 00 00  b0 00 00 00 00 00 00 00  |................|
00000050  b0 00 40 00 00 00 00 00  b0 00 40 00 00 00 00 00  |..@.......@.....|
00000060  14 00 00 00 00 00 00 00  14 00 00 00 00 00 00 00  |................|
00000070  08 00 00 00 00 00 00 00  51 e5 74 64 06 00 00 00  |........Q.td....|
00000080  00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  |................|
*
000000a0  00 00 00 00 00 00 00 00  08 00 00 00 00 00 00 00  |................|
000000b0  03 00 00 94 a8 0b 80 52  01 00 00 d4 40 05 80 52  |.......R....@..R|
000000c0  c0 03 5f d6                                       |.._.|
000000c4


## Using my personal generic loader

The loader already exists and is described in `monitor.ipynb`. It is in `/usr/local/bin/` so that other programs can use it as a generic loader.

In [103]:
! objcopy -O binary a.out a.bin

In [104]:
! load a.bin ; echo $?

42


In [105]:
! wc -c a.bin

20 a.bin


The file is only 12 bytes long and contains only the executable code, without the ELF overhead.

In [106]:
! hexdump -C a.bin

00000000  03 00 00 94 a8 0b 80 52  01 00 00 d4 40 05 80 52  |.......R....@..R|
00000010  c0 03 5f d6                                       |.._.|
00000014


In [109]:
! objdump -D -b binary -m aarch64 a.bin


a.bin:     file format binary


Disassembly of section .data:

0000000000000000 <.data>:
   0:	94000003 	bl	0xc
   4:	52800ba8 	mov	w8, #0x5d                  	// #93
   8:	d4000001 	svc	#0x0
   c:	52800540 	mov	w0, #0x2a                  	// #42
  10:	d65f03c0 	ret


From a Fortran program we were able to obtain a bare-metal executable AArch64 binary through a custom loader, 20 bytes in size.